In [1]:
import csv
#import necessary packages
import pandas as pd
#import spotify api wrapper and authorization package
import spotipy
from spotipy.oauth2 import SpotifyOAuth
#import language processing packages
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import re

In [2]:
#create a search function for a keyword
def search(key, df):
    key = key.lower()
    char = ',./\|;: '
    searchList = []
    count = 0
    
    fullString = re.sub('[^A-Za-z0-9]+', ' ', key)
    cutString = fullString.split()
    
    for keyword in cutString:
        for title,desc, episode in zip(df['title_analyzed'], df['description_analyzed'], df['title']):
            if all(word in title or word in desc for word in cutString):
            #if (keyword in title) or (keyword in desc):
                count = count + 1
                searchList.append(episode)
        df3 = pd.DataFrame()
        for item in searchList:
            line = df[df['title'] == item]
            df3 = df3.append(line)
        return df3

In [3]:
def getPodcastEpisodes(url):
    #setup spotify credentials
    creds = open('creds.csv', 'r').read()
    creds.split(',')
    
    cid = creds.split(',')[0]
    secret = creds.split(',')[1]
    uri = creds.split(',')[2]
    username = creds.split(',')[3]
    #this scope is to read personal library [can be changed as needed]
    scope = "user-library-read"
    

    #trial test to read out 20 most recent starred tracks
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id = cid, client_secret = secret, redirect_uri = uri, username=username, scope=scope))
    #url = 'https://open.spotify.com/show/07SjDmKb9iliEzpNcN2xGD'

    index = 0
    episodes = []
    val = True

    #Get a list of dictionaries of all the available episodes on Spotify
    while val:
        eps = sp.show_episodes(url, offset=index)
        if len(eps['items']) > 0: 
            episodes.append(eps)
            index += 50
        else:
            val = False


    #create dictionary for episode title, date, and description
    dummy = {}
    index = 0
    for dc in episodes:
        for item in dc['items']:
            title = item['name']
            date = item['release_date']
            desc = item['description']

            dummy[index] = ({'title': title, 'date': date, 'description': desc})
            index+=1
    #convert dictionary to a pandas dataframe
    df = pd.DataFrame(dummy)
    df2 = df.T
    
    wordList = []
    descList = []
    for title, desc in zip(df2['title'], df2['description']):
        wordList.append(word_tokenize(title.lower()))
        descList.append(word_tokenize(desc.lower()))

    df2['title_analyzed'] = wordList
    df2['description_analyzed'] = descList
    
    return df2